In [56]:
import os
import pytz
from datetime import datetime

In [57]:
from pathlib import Path

In [58]:
from dotenv import load_dotenv

In [59]:
# Dynamically locate the project root (assumes "nbs" is a known subdirectory)
current_dir = Path(os.getcwd()).resolve()
project_root = current_dir.parent  # Go up one level to the root directory
dotenv_path = project_root / ".env"

In [60]:
# Load the `.env` file
if dotenv_path.exists():
    load_dotenv(dotenv_path)
    print(f".env loaded from {dotenv_path}")
else:
    print(f"Warning: .env file not found at {dotenv_path}")

# Access DATABASE_URL
database_url = os.getenv("DATABASE_URL", "sqlite:///default.db")  # Provide a fallback
alpha_vantage_api = os.getenv("ALPHA_VANTAGE_API_KEY")
print("ALPHA_VANTAGE_API_KEY:", alpha_vantage_api)

.env loaded from /Users/mac/PycharmProjects/Stock-Trading-Bot/.env
ALPHA_VANTAGE_API_KEY: X6ZWES4Q9CJ66QV6


In [61]:
print("Current working directory:", os.getcwd())

Current working directory: /Users/mac/PycharmProjects/Stock-Trading-Bot/nbs


In [62]:
ALPHA_VANTAGE_API = os.getenv("ALPHA_VANTAGE_API_KEY")

In [63]:
import requests

In [64]:
params = {
    "api_key" : ALPHA_VANTAGE_API,
    "ticker" : "AAPL",
    "function" : "TIME_SERIES_DAILY"
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=5min&apikey={api_key}'.format(**params)
r = requests.get(url)
data = r.json()

In [65]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (Daily)'

In [66]:
results = data[dataset_key]
results

{'2024-11-20': {'1. open': '228.0600',
  '2. high': '229.9300',
  '3. low': '225.8900',
  '4. close': '229.0000',
  '5. volume': '35169566'},
 '2024-11-19': {'1. open': '226.9800',
  '2. high': '230.1600',
  '3. low': '226.6600',
  '4. close': '228.2800',
  '5. volume': '36211774'},
 '2024-11-18': {'1. open': '225.2500',
  '2. high': '229.7400',
  '3. low': '225.1700',
  '4. close': '228.0200',
  '5. volume': '44686020'},
 '2024-11-15': {'1. open': '226.4000',
  '2. high': '226.9200',
  '3. low': '224.2700',
  '4. close': '225.0000',
  '5. volume': '47923696'},
 '2024-11-14': {'1. open': '225.0200',
  '2. high': '228.8700',
  '3. low': '225.0000',
  '4. close': '228.2200',
  '5. volume': '44923941'},
 '2024-11-13': {'1. open': '224.0100',
  '2. high': '226.6500',
  '3. low': '222.7600',
  '4. close': '225.1200',
  '5. volume': '48566217'},
 '2024-11-12': {'1. open': '224.5500',
  '2. high': '225.5900',
  '3. low': '223.3550',
  '4. close': '224.2300',
  '5. volume': '40398299'},
 '2024

In [67]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2024-11-20'

In [68]:
timestamp_format = '%Y-%m-%d'
eastern = pytz.timezone("US/Eastern")
utc = pytz.utc
timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
timestamp

datetime.datetime(2024, 11, 20, 5, 0, tzinfo=<UTC>)

In [69]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '228.0600',
 '2. high': '229.9300',
 '3. low': '225.8900',
 '4. close': '229.0000',
 '5. volume': '35169566'}

In [71]:
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result):
    timestamp_format = '%Y-%m-%d'
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': Decimal(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }

In [72]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('228.0600'),
 'close_price': Decimal('229.0000'),
 'high_price': Decimal('229.9300'),
 'low_price': Decimal('225.8900'),
 'number_of_trades': None,
 'volume': Decimal('35169566'),
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 11, 20, 5, 0, tzinfo=<UTC>)}

In [73]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('228.0600'), 'close_price': Decimal('229.0000'), 'high_price': Decimal('229.9300'), 'low_price': Decimal('225.8900'), 'number_of_trades': None, 'volume': Decimal('35169566'), 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 20, 5, 0, tzinfo=<UTC>)}
{'open_price': Decimal('226.9800'), 'close_price': Decimal('228.2800'), 'high_price': Decimal('230.1600'), 'low_price': Decimal('226.6600'), 'number_of_trades': None, 'volume': Decimal('36211774'), 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 19, 5, 0, tzinfo=<UTC>)}
{'open_price': Decimal('225.2500'), 'close_price': Decimal('228.0200'), 'high_price': Decimal('229.7400'), 'low_price': Decimal('225.1700'), 'number_of_trades': None, 'volume': Decimal('44686020'), 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 18, 5, 0, tzinfo=<UTC>)}
{'open_price': Decimal('226.4000'), 'close_price': Decimal('225.0000'), 'high_price': Decimal('226.9200'), 'low_price': Deci